In [8]:
# works on Python 3.8+
import inspect, sys, typing as t, types

def get_var_annotation(
    value=None, *,               # an object you have (optional)
    name: str | None = None,     # its name (if you know it)
    module: str | types.ModuleType | None = None,  # module where it's defined
    owner=None,                  # class or instance for attributes
    attr: str | None = None,     # attribute name on the owner
    default=None,
):
    """Return the annotation object, or `default` if it can't be found.

    Handles:
      - module globals: x: int = ...
      - class attrs:   class C: y: list[int]
      - instance attrs, if annotated on the class
      - functions/methods: returns a dict of param/return hints
    """
    # 1) Function or method -> return its full hint mapping
    if inspect.isfunction(value) or inspect.ismethod(value):
        return t.get_type_hints(value, include_extras=True)

    # 2) Class/instance attribute annotation (lives on the class)
    if owner is not None and attr is not None:
        cls = owner if inspect.isclass(owner) else owner.__class__
        try:
            return t.get_type_hints(cls, include_extras=True).get(attr, default)
        except Exception:
            return getattr(cls, "__annotations__", {}).get(attr, default)

    # 3) Module-level variable annotation
    if module is None:
        # default to the caller's module
        frame = inspect.currentframe().f_back
        module = inspect.getmodule(frame)
    elif isinstance(module, str):
        module = sys.modules[module]

    if name is None and value is not None and module is not None:
        # try to discover a name in the module that refers to this object
        for n, v in module.__dict__.items():
            if v is value:
                name = n
                break

    if name is not None and module is not None:
        try:
            # resolves forward refs & Annotated
            return t.get_type_hints(module, include_extras=True).get(name, default)
        except Exception:
            # fallback: raw annotations (may be strings)
            return getattr(module, "__annotations__", {}).get(name, default)

    return default


In [9]:
from typing import Annotated
rate: Annotated[float, "kW"] = 1.2

get_var_annotation(name="rate", module=__name__)

typing.Annotated[float, 'kW']

In [15]:
from tensorspecs import TensorLike


rate: TensorLike["a b c", float] = 1.2

todo = get_var_annotation(rate, module=__name__)
dir(todo)

['__args__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__instancecheck__',
 '__iter__',
 '__le__',
 '__lt__',
 '__metadata__',
 '__module__',
 '__mro_entries__',
 '__ne__',
 '__new__',
 '__or__',
 '__origin__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasscheck__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_determine_new_args',
 '_inst',
 '_make_substitution',
 '_name',
 '_nparams',
 '_paramspec_tvars',
 'copy_with',
 'get',
 'items',
 'keys',
 'values']

In [16]:
import typing

typing.get_args(todo)

(typing.Mapping, ('a b c', float))

In [25]:
import inspect

inspect.get_annotations(__name__)

TypeError: '__main__' is not a module, class, or callable.

In [20]:

def todo(a: ...):
    # tensorspecs.shape(a)

    pass

In [27]:
from robotodo.utils.geometry import PolygonMesh

PolygonMesh.__annotations__
PolygonMesh.triangulate.__annotations__

{'copy': bool}

In [30]:
# import einops
# import numpy as np

# einops.parse_shape(np.zeros([2, 3, 5, 7]))